In [2]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC
import warnings


It is highly recommended to configure Git before using DataLad. Set both 'user.name' and 'user.email' configuration variables.


In [3]:
storage = HDF5FeatureStorage(uri='./data/AOMIC_Parcels_sch100x7.hdf5')

In [4]:
df_parcellations = storage.read_df('VBM_GM_Schaefer100x7_mean_aggregation')
df_histograms= storage.read_df('VBM_GM_Histogram_100bins_IXI_hist')
df_demographics = pd.read_csv('./data/participants.tsv',sep='\t')
df_demographics.rename(columns={"participant_id": "subject"}, inplace=True)

In [5]:
df_parcellations.dropna(inplace=True)
df_histograms.dropna(inplace=True)


In [6]:
df_parcellations.columns = df_parcellations.columns.astype(str)
df_histograms.columns = df_histograms.columns.astype(str)

# List of columns for each dataframe
X_parcellations = list(df_parcellations.columns)
X_histograms = list(df_histograms.columns)

# Merge with df_demographics on 'subject'
df_full_parcellations = df_parcellations.merge(df_demographics, on="subject")
df_full_histograms = df_histograms.merge(df_demographics, on="subject")


In [11]:
# For SVM (Support Vector Machine)
creator_svm = PipelineCreator(problem_type="classification")
creator_svm.add("zscore")
creator_svm.add(
    "svm",
    C=(0.001, 100, "log-uniform"),
)

search_params_svm = {
    "kind": "optuna",
    "cv": 4
}

scoring = ["balanced_accuracy", "accuracy"]

# SVM on histograms
scores_hists_svm, model_hists_svm, inspector_hists_svm = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params_svm,
    model=creator_svm,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)

# SVM on parcellations
scores_schaefer_svm, model_schaefer_svm, inspector_schaefer_svm = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params_svm,
    model=creator_svm,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)





/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/pipeline/pipeline_creator.py:1041: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the futur

In [12]:
scores_schaefer_svm

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.918239,0.009384,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.840558,0.960547,0.836283,0.960059,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.900648,0.009733,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.783790,0.907967,0.778761,0.908284,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.938295,0.009372,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.827519,0.995583,0.826667,0.995569,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.909972,0.009176,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.790890,0.957960,0.791111,0.957164,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [13]:
scores_hists_svm

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.975370,0.009428,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.746690,0.846556,0.743363,0.847633,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.842501,0.010060,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.710516,0.751374,0.699115,0.760355,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.885955,0.009018,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.732679,0.854739,0.737778,0.855244,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.802027,0.009434,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.675174,0.786308,0.675556,0.790251,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [8]:
# For Random Forest
creator_rf = PipelineCreator(problem_type="classification")
creator_rf.add("zscore")
creator_rf.add(
    "rf",
)

search_params_rf = {
    "kind": "optuna",
    "cv": 4
}

# Random Forest on histograms
scores_hists_rf, model_hists_rf, inspector_hists_rf = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params_rf,
    model=creator_rf,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)

# Random Forest on parcellations
scores_schaefer_rf, model_schaefer_rf, inspector_schaefer_rf = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params_rf,
    model=creator_rf,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['LH_Vis_1', 'LH_Vis_2', 'LH_Vis_3', 'LH_Vis_4', 'LH_Vis_5', 'LH_Vis

In [10]:
scores_schaefer_rf

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.342736,0.006852,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.764344,1.0,0.765487,1.0,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.343481,0.006642,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.766983,1.0,0.761062,1.0,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.342093,0.006731,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.788517,1.0,0.791111,1.0,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.348962,0.006687,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.737555,1.0,0.737778,1.0,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [9]:
scores_hists_rf

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.462801,0.007932,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.686318,1.0,0.685841,1.0,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.331367,0.006728,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.696497,1.0,0.685841,1.0,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.335238,0.007177,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.680717,1.0,0.688889,1.0,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.331795,0.006803,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.630887,1.0,0.631111,1.0,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [15]:
# For Extra Trees Classifier
creator_et = PipelineCreator(problem_type="classification")
creator_et.add("zscore")
creator_et.add(
    "et",
)

search_params_et = {
    "kind": "optuna",
    "cv": 4
}

scoring = ["balanced_accuracy", "accuracy"]

# Extra Trees on histograms
scores_hists_et, model_hists_et, inspector_hists_et = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params_et,
    model=creator_et,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)

# Extra Trees on parcellations
scores_schaefer_et, model_schaefer_et, inspector_schaefer_et = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params_et,
    model=creator_et,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)


/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['LH_Vis_1', 'LH_Vis_2', 'LH_Vis_3', 'LH_Vis_4', 'LH_Vis_5', 'LH_Vis

In [16]:
scores_schaefer_et

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.114881,0.007556,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.795870,1.0,0.796460,1.0,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.120438,0.007795,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.743187,1.0,0.734513,1.0,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.114420,0.007344,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.798813,1.0,0.804444,1.0,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.113724,0.007376,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.719580,1.0,0.720000,1.0,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [17]:
scores_hists_et

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.130394,0.009151,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.687579,1.0,0.690265,1.0,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,0.121063,0.007431,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.696969,1.0,0.685841,1.0,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,0.120920,0.007440,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.706638,1.0,0.715556,1.0,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,0.120398,0.007735,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.639697,1.0,0.640000,1.0,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [25]:
# For Gradient Boosting Classifier
creator_gb = PipelineCreator(problem_type="classification")
creator_gb.add("zscore")
creator_gb.add(
    "gradientboost",
)

search_params_gb = {
    "kind": "optuna",
    "cv": 4
}

scoring = ["balanced_accuracy", "accuracy"]

# Gradient Boosting on histograms
scores_hists_gb, model_hists_gb, inspector_hists_gb = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params_gb,
    model=creator_gb,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)

# Gradient Boosting on parcellations
scores_schaefer_gb, model_schaefer_gb, inspector_schaefer_gb = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params_gb,
    model=creator_gb,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring=scoring,
)


/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['LH_Vis_1', 'LH_Vis_2', 'LH_Vis_3', 'LH_Vis_4', 'LH_Vis_5', 'LH_Vis

In [26]:
scores_schaefer_gb

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,1.432737,0.004237,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.837721,0.997038,0.836283,0.997041,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,1.423954,0.005536,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.791251,1.000000,0.787611,1.000000,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,1.446053,0.004122,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.840359,1.000000,0.844444,1.000000,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,1.440650,0.003774,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.764302,0.998433,0.764444,0.998523,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [27]:
scores_hists_gb

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,1.396482,0.006703,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.666535,0.980508,0.663717,0.980769,676,226,0,0,bc7087515161a73a5a6aff57863f3803
1,1.388387,0.003819,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.701602,0.981227,0.694690,0.982249,676,226,0,1,bc7087515161a73a5a6aff57863f3803
2,1.370233,0.005115,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.667636,0.979227,0.680000,0.979321,677,225,0,2,bc7087515161a73a5a6aff57863f3803
3,1.371940,0.004955,"(SetColumnTypes(X_types={}), StandardScaler(),...",0.644161,0.979965,0.644444,0.980798,677,225,0,3,bc7087515161a73a5a6aff57863f3803


In [29]:
scores_schaefer_svm['model'] = 'AOMIC_Schaefer_SVM'
scores_hists_svm['model'] = 'AOMIC_Histograms_SVM'

scores_schaefer_rf['model'] = 'AOMIC_Schaefer_RF'
scores_hists_rf['model'] = 'AOMIC_Histograms_RF'

scores_schaefer_et['model'] = 'AOMIC_Schaefer_ET'
scores_hists_et['model'] = 'AOMIC_Histograms_ET'

scores_schaefer_gb['model'] = 'AOMIC_Schaefer_GB'
scores_hists_gb['model'] = 'AOMIC_Histograms_GB'


In [30]:
plot_scores(scores_schaefer_svm,scores_hists_svm,scores_schaefer_rf,scores_hists_rf,scores_schaefer_et,scores_hists_et,scores_schaefer_gb,scores_hists_gb)

BokehModel(combine_events=True, render_bundle={'docs_json': {'58039fcd-7bda-4ff9-841f-626be0ffd843': {'version…